In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import yfinance as yf
import json
import numpy as np
import pandas as pd

In [ ]:
tickers = yf.Tickers("MSFT AMZN")

In [ ]:
data = yf.download("MSFT AMZN", start="2000-01-01", end="2022-01-01")["Adj Close"]
data.plot()

In [ ]:
def to_relative(series: pd.Series) -> pd.Series:
    return series / series.iloc[0]


data.apply(to_relative).plot()

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
def abs_start_end_change(series: pd.Series) -> float:
    return series.iloc[-1] - series.iloc[0]


def rel_start_end_change(series: pd.Series) -> float:
    return (abs_start_end_change(series) / series.iloc[0]) * 100


def max_fall(series: pd.Series) -> float:
    return np.min(
        [
            (np.min(series.iloc[i:] - np.max(series.iloc[:i])))
            / np.max(series.iloc[:i])
            for i in range(1, len(series))
        ]
    )


def max_rise(series: pd.Series) -> float:
    return np.max(
        [
            (np.max(series.iloc[i:] - np.min(series.iloc[:i])))
            / np.min(series.iloc[:i])
            for i in range(1, len(series))
        ]
    )

In [ ]:
data.agg([max_fall, max_rise])

In [ ]:
pd.concat(
    (
        data.describe(),
        data.agg([abs_start_end_change, rel_start_end_change, max_fall, max_rise]),
    )
).round(2).transpose().sort_values(by="rel_start_end_change", ascending=False)

In [ ]:
data.agg(
    ["count", "min", "mean", "max", abs_start_end_change, rel_start_end_change]
).round(2).transpose()

In [ ]:
(339.92 - 11.40) / 11.40

In [ ]:
type(np.mean([1, 2, 3]))

In [ ]:
data = yf.download("BTC-USD", start="2008-01-01", end="2022-01-01")["Adj Close"]
data.plot()
data = yf.download("ETH-USD", start="2008-01-01", end="2022-01-01")["Adj Close"]
data.plot()

In [ ]:
yf.download("BTC-USD", start="2008-01-01", end="2022-01-01")

In [ ]:
yf.download("ETH-USD", start="2008-01-01", end="2022-01-01")

In [ ]:
info = yf.Ticker("0P00011HBM.F").info

In [ ]:
info.keys()

In [ ]:
info["holdings"]

In [ ]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get all stock info (slow)
msft.info
# fast access to subset of stock info (opportunistic)
msft.fast_info

# get historical market data
hist = msft.history(period="1mo")

# show meta information about the history (requires history() to be called first)
msft.history_metadata

# show actions (dividends, splits, capital gains)
msft.actions
msft.dividends
msft.splits
msft.capital_gains  # only for mutual funds & etfs

# show share count
# - yearly summary:
msft.shares
# - accurate time-series count:
msft.get_shares_full(start="2022-01-01", end=None)

# show financials:
# - income statement
msft.income_stmt
msft.quarterly_income_stmt
# - balance sheet
msft.balance_sheet
msft.quarterly_balance_sheet
# - cash flow statement
msft.cashflow
msft.quarterly_cashflow
# see `Ticker.get_income_stmt()` for more options

# show holders
msft.major_holders
msft.institutional_holders
msft.mutualfund_holders

# show earnings
msft.earnings
msft.quarterly_earnings

# show sustainability
msft.sustainability

# show analysts recommendations
msft.recommendations
msft.recommendations_summary
# show analysts other work
msft.analyst_price_target
msft.revenue_forecasts
msft.earnings_forecasts
msft.earnings_trend

# show next event (earnings, etc)
msft.calendar

# Show future and historic earnings dates, returns at most next 4 quarters and last 8 quarters by default.
# Note: If more are needed use msft.get_earnings_dates(limit=XX) with increased limit argument.
msft.earnings_dates

# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
msft.isin

# show options expirations
msft.options

# show news
msft.news

# get option chain for specific expiration
opt = msft.option_chain("YYYY-MM-DD")
# data available via: opt.calls, opt.puts